### 4. Receiving Leaders

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import math
from datetime import date
import re

In [2]:
# rushing url
espn_url = 'https://www.espn.co.uk/college-football/stats/player/_/view/offense/stat/receiving'

In [3]:
# get raw data
page = requests.get(espn_url)
soup = BeautifulSoup(page.content, "html.parser")
results = soup.find(id="fittPageContainer") # found by inspecting html

In [4]:
raw_output = [i.text for i in results.find_all("td", class_="Table__TD")]

In [5]:
n_rows = 50

# get rb names
players = []
for i in range(n_rows*2):
    if i % 2 != 0:
        players.append(raw_output[i])

school = []
player_clean = []

# split out the school from the surname of the QB
# could probably use REGEX to improve this, but it's fine
for p in players:
    if p[len(p)-4].isupper():
        # 4 letter team abbrv
        player_clean.append(p[:len(p)-4])
        school.append(p[len(p)-4:])
    elif p[len(p)-3].isupper():
        # 3 letter abbrv
        player_clean.append(p[:len(p)-3])
        school.append(p[len(p)-3:])
    else:
        # 2 letter abbrv
        player_clean.append(p[:len(p)-2])
        school.append(p[len(p)-2:])
        
# get other stats
raw_stats = raw_output[n_rows*2:]
table = pd.DataFrame(np.reshape(raw_stats, (int(len(raw_stats)/6), 6)), 
                     columns = ['POS','REC','YDS','AVG','LNG','TD'])
table.insert(0, 'NAME', player_clean)
table.insert(1, 'ABBRV', school)

In [6]:
table.head()

,NAME,ABBRV,POS,REC,YDS,AVG,LNG,TD
0,Deven Thompkins,USU,WR,38,639,16.8,75,4
1,Drake London,USC,WR,39,540,13.8,39,3
2,Dontayvion Wicks,UVA,WR,24,535,22.3,49,5
3,Calvin Austin II,IMEM,WR,27,533,19.7,75,7
4,Dontay Demus Jr.,MD,WR,28,507,18.1,66,3


In [7]:
# save out as dataset
todays_date = str(date.today())
table.to_csv('data/'+todays_date+'-wr-records.csv', index=False)